In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
from tqdm import tqdm_notebook

In [3]:
ws = open(r'C:\Users\venug\Downloads\Conference Calendar - ESMO Breast Cancer Virtual Meeting 2020.html',encoding = 'utf8')
data = ws.read()

In [4]:
data = BeautifulSoup(data,'lxml')

In [26]:
response = requests.get(data.find_all('h4',class_ = 'session-title card-title')[0].find('a')['href'])
response.text

'<!doctype html>\n<html lang="en">\n<head>\n<meta charset="UTF-8">\n<title>Conference Calendar - ESMO Breast Cancer Virtual Meeting 2020</title>\n<meta name="viewport" content="width=device-width, initial-scale=1.0, user-scalable=0"/>\n<meta name="description" content=""/>\n<meta name="author" content="CTI Meeting Technology"/>\n<link rel="shortcut icon" href="/images/banner/favicon.ico">\n<link rel="stylesheet" type="text/css" href="/stylesheets/deliver_less.php?hash=lessphp_2c20a0852e504fd420421a9fb8ad24a34fec754d.css"/>\n<style>\n.st-1{color: #000 !important;background-color: #83ff33 !important}.st-1-bg-fill{--opacity: 70;background-image: linear-gradient(to right, rgba(131,255,51,0.7), rgba(131,255,51,0.7))}.st-1-bg-fill a{color: #286600 !important}.st-1-bg-fill:hover{background-image: linear-gradient(to right, rgba(131,255,51,0.75), rgba(131,255,51,0.75))}.st-1-bg{background-image: linear-gradient(to right, rgba(131,255,51,0.4), rgba(0,0,0,0))}.st-1-fa{color: #83ff33}.st-1-border{

In [33]:
data.find_all('h4',class_ = 'session-title card-title')

[<h4 class="session-title card-title"><a class="item-expand-action expand" href="https://cslide.ctimeetingtech.com/breast2020/attendee/confcal/show/session/65">e-Posters <small>(ID 65)</small></a></h4>,
 <h4 class="session-title card-title"><a class="item-expand-action expand" href="https://cslide.ctimeetingtech.com/breast2020/attendee/confcal/show/session/1">Opening and welcome <small>(ID 1)</small></a></h4>,
 <h4 class="session-title card-title"><a class="item-expand-action expand" href="https://cslide.ctimeetingtech.com/breast2020/attendee/confcal/show/session/2">ESMO Breast Cancer Award <small>(ID 2)</small></a></h4>,
 <h4 class="session-title card-title"><a class="item-expand-action expand" href="https://cslide.ctimeetingtech.com/breast2020/attendee/confcal/show/session/7">Metastatic breast cancer: HER2+ <small>(ID 7)</small></a></h4>,
 <h4 class="session-title card-title"><a class="item-expand-action expand" href="https://cslide.ctimeetingtech.com/breast2020/attendee/confcal/show

In [10]:
df = pd.DataFrame()

In [11]:
for i in tqdm_notebook(data.find_all('h4',class_ = 'session-title card-title')):
    response = requests.get(i.find('a')['href'])
    sessData = BeautifulSoup(response.text,'lxml')
    sessDict = {}
    sessDict['Session Title'] = sessData.find('h4',class_ = 'session-title card-title').find('a').text
    sessDict['Date'] = sessData.find('div',class_ = 'property-container internal_date')
    sessDict['Time'] = sessData.find('div',class_ = 'property-container internal_time')
    sessDict['Location'] = sessData.find('div',class_ = 'property-container internal_location')
    sessDict['Moderators'] = sessData.find('div',class_ = 'property-container internal_moderators').text
    presentations = sessData.find_all('div',class_ = 'card presentation')
    if(len(presentations)==0):
            df = df.append(sessDict,ignore_index = True)
    else:
        for pres in tqdm_notebook(presentations):
            presDict = sessDict.copy()
            details = pres.find('div',class_ = 'card-block')
            presDict['Presentation Title'] = details.find('h4',class_ = 'card-title').text
            intDetails = details.find('div',class_ = 'details property-auto-width')
            properties = intDetails.find_all('div',class_ = re.compile('property-container'))
            for i in properties:
                try:
                    presDict[i.find('div',class_ = 'info').text] = i.find('div',class_ = 'property').text
                except:
                    pass
            df = df.append(presDict,ignore_index = True)


C:\Users\venug\anaconda3\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


C:\Users\venug\anaconda3\lib\site-packages\ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [12]:
df

,Date,Lecture Time,Location,Moderators,Presentation Title,Session Name,Session Title,Time,Presentation Number,Speakers
0,09.05.2020,12:00 - 12:00,"Webcast - not visible, Virtual Meeting, Vienna...",\n,Biomarkers and translational research and prec...,ePoster (ID 19),e-Posters (ID 65),12:00 - 13:00,NaN,NaN
1,09.05.2020,12:00 - 12:00,"Webcast - not visible, Virtual Meeting, Vienna...",\n,5P - Circulating tumor DNA (ctDNA) dynamics us...,ePoster (ID 19),e-Posters (ID 65),12:00 - 13:00,5P,"\nOlga Martínez-Sáez (Barcelona, Spain)\n\n"
2,09.05.2020,12:00 - 12:00,"Webcast - not visible, Virtual Meeting, Vienna...",\n,6P - Circulating tumor cells (CTCs) as biomark...,ePoster (ID 19),e-Posters (ID 65),12:00 - 13:00,6P,"\nFrancesca Galardi (Prato, Italy)\n\n"
3,09.05.2020,12:00 - 12:00,"Webcast - not visible, Virtual Meeting, Vienna...",\n,7P - Prognostic value of the immune infiltrati...,ePoster (ID 19),e-Posters (ID 65),12:00 - 13:00,7P,"\nGuoxing Wan (Shiyan, China)\n\n"
4,09.05.2020,12:00 - 12:00,"Webcast - not visible, Virtual Meeting, Vienna...",\n,8P - Characterization of immune microenvironme...,ePoster (ID 19),e-Posters (ID 65),12:00 - 13:00,8P,"\nGaia Griguolo (Padova, Italy)\n\n"
...,...,...,...,...,...,...,...,...,...,...
440,24.05.2020,17:45 - 18:05,"Channel 1, Virtual Meeting, Vienna, Austria","\nChairs\n\nFabrice André (Villejuif, France)\...",How to predict sensitivity to anti-PD(L)1? (ID...,Precision medicine in metastatic triple negati...,Precision medicine in metastatic triple negati...,17:45 - 19:15,NaN,"\nSherene Loi (Melbourne, VIC, Australia)\n\n"
441,24.05.2020,18:05 - 18:25,"Channel 1, Virtual Meeting, Vienna, Austria","\nChairs\n\nFabrice André (Villejuif, France)\...",What is the target population of PARP inhibito...,Precision medicine in metastatic triple negati...,Precision medicine in metastatic triple negati...,17:45 - 19:15,NaN,"\nCristina Saura Manich (Barcelona, Spain)\n\n"
442,24.05.2020,18:25 - 18:45,"Channel 1, Virtual Meeting, Vienna, Austria","\nChairs\n\nFabrice André (Villejuif, France)\...",Dissecting PI3K/AKT/mTOR pathway in mTNBC (ID 42),Precision medicine in metastatic triple negati...,Precision medicine in metastatic triple negati...,17:45 - 19:15,NaN,"\nRebecca Kristeleit (London, United Kingdom)\n\n"
443,24.05.2020,18:45 - 19:05,"Channel 1, Virtual Meeting, Vienna, Austria","\nChairs\n\nFabrice André (Villejuif, France)\...",How to integrate predictors in a single algori...,Precision medicine in metastatic triple negati...,Precision medicine in metastatic triple negati...,17:45 - 19:15,NaN,"\nJorge S. Reis-Filho (New York, NY, United St..."


In [13]:
df['Moderators'] = df['Moderators'].str.replace('\n',' ')

In [14]:
df['Speakers'] = df['Speakers'].str.replace('\n',' ')

In [15]:
df

,Date,Lecture Time,Location,Moderators,Presentation Title,Session Name,Session Title,Time,Presentation Number,Speakers
0,09.05.2020,12:00 - 12:00,"Webcast - not visible, Virtual Meeting, Vienna...",,Biomarkers and translational research and prec...,ePoster (ID 19),e-Posters (ID 65),12:00 - 13:00,NaN,NaN
1,09.05.2020,12:00 - 12:00,"Webcast - not visible, Virtual Meeting, Vienna...",,5P - Circulating tumor DNA (ctDNA) dynamics us...,ePoster (ID 19),e-Posters (ID 65),12:00 - 13:00,5P,"Olga Martínez-Sáez (Barcelona, Spain)"
2,09.05.2020,12:00 - 12:00,"Webcast - not visible, Virtual Meeting, Vienna...",,6P - Circulating tumor cells (CTCs) as biomark...,ePoster (ID 19),e-Posters (ID 65),12:00 - 13:00,6P,"Francesca Galardi (Prato, Italy)"
3,09.05.2020,12:00 - 12:00,"Webcast - not visible, Virtual Meeting, Vienna...",,7P - Prognostic value of the immune infiltrati...,ePoster (ID 19),e-Posters (ID 65),12:00 - 13:00,7P,"Guoxing Wan (Shiyan, China)"
4,09.05.2020,12:00 - 12:00,"Webcast - not visible, Virtual Meeting, Vienna...",,8P - Characterization of immune microenvironme...,ePoster (ID 19),e-Posters (ID 65),12:00 - 13:00,8P,"Gaia Griguolo (Padova, Italy)"
...,...,...,...,...,...,...,...,...,...,...
440,24.05.2020,17:45 - 18:05,"Channel 1, Virtual Meeting, Vienna, Austria","Chairs Fabrice André (Villejuif, France)",How to predict sensitivity to anti-PD(L)1? (ID...,Precision medicine in metastatic triple negati...,Precision medicine in metastatic triple negati...,17:45 - 19:15,NaN,"Sherene Loi (Melbourne, VIC, Australia)"
441,24.05.2020,18:05 - 18:25,"Channel 1, Virtual Meeting, Vienna, Austria","Chairs Fabrice André (Villejuif, France)",What is the target population of PARP inhibito...,Precision medicine in metastatic triple negati...,Precision medicine in metastatic triple negati...,17:45 - 19:15,NaN,"Cristina Saura Manich (Barcelona, Spain)"
442,24.05.2020,18:25 - 18:45,"Channel 1, Virtual Meeting, Vienna, Austria","Chairs Fabrice André (Villejuif, France)",Dissecting PI3K/AKT/mTOR pathway in mTNBC (ID 42),Precision medicine in metastatic triple negati...,Precision medicine in metastatic triple negati...,17:45 - 19:15,NaN,"Rebecca Kristeleit (London, United Kingdom)"
443,24.05.2020,18:45 - 19:05,"Channel 1, Virtual Meeting, Vienna, Austria","Chairs Fabrice André (Villejuif, France)",How to integrate predictors in a single algori...,Precision medicine in metastatic triple negati...,Precision medicine in metastatic triple negati...,17:45 - 19:15,NaN,"Jorge S. Reis-Filho (New York, NY, United Sta..."


In [16]:
df.to_excel('ESMO.xls')